In [1]:
import scanpy as sc
import anndata as ad
from scib_metrics.benchmark import Benchmarker
from scib_metrics.benchmark import BatchCorrection

from scib_metrics_new.benchmarked_plot_results_table import plot_results_table_new

import pandas as pd
import numpy as np
import pickle

/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

In [3]:
DATA_NAME = 'BMMC'

# datasets = [DATA_NAME, DATA_NAME + '_rm25pct', DATA_NAME + '_rm50pct', 
#             DATA_NAME + '_rm75pct', DATA_NAME + '_rm100pct']

datasets = [DATA_NAME,]

DATA_PATH_list = ['outputs/inte_outputs/' + item + '/' for item in datasets]

OUTPUT_PATH = 'outputs/inte_outputs/'

scDiffusion_PATH_list = ['/projectnb/czproj/Algorithms/ycliu/Pan_proj/outputs/inte_outputs/' + item + '/' for item in datasets]


# Function: Load data

In [4]:
def load_adata(path, path_sd):    
    adata = sc.read_h5ad(path +'Harmony_integration.h5ad')
    
    for package in ['Harmony', 'scVI', 'Combat', 'Scanorama']:
        df = pd.read_csv(path + package + '_matrix.csv', sep='\t', header=None)
        adata.obsm[package] = np.array(df)
        
    df = pd.read_csv(path_sd + 'scDiffusion_matrix.csv', sep='\t', header=None)
    adata.obsm['scDiffusion'] = np.array(df)
        
    
    df = pd.read_csv(path+'SeuratV4_matrix_with_index.csv', index_col=0)
    df = df.reindex(adata.obs.index)
    adata.obsm['Seurat'] = np.array(df)

    df = pd.read_csv(path+'fastMNN_matrix_with_index.csv', index_col=0)
    df = df.reindex(adata.obs.index)
    adata.obsm['fastMNN'] = np.array(df)
    
    adata.obsm['Unintegrated'] = adata.obsm['X_pca']
    
    return adata

# Function: Evaluation

In [5]:
batch_corr = BatchCorrection(pcr_comparison=False)

def evaluate_integration(adata, batch_corr=batch_corr):
    bm = Benchmarker(
        adata,
        batch_key="batch",
        label_key="labels",
        embedding_obsm_keys=['scDiffusion', "Harmony", "scVI", 'Combat', 'Scanorama', 
                             'Seurat', 'fastMNN', 'Unintegrated'],
        pre_integrated_embedding_obsm_key='Unintegrated',
        batch_correction_metrics=batch_corr,
        n_jobs=1,
    )
    bm.benchmark()
    
    return bm

# Evaluation

In [ ]:
bm_list = []

for path, path_sd in zip(DATA_PATH_list, scDiffusion_PATH_list):
    
    adata = load_adata(path, path_sd)
    bm = evaluate_integration(adata)
    
    file_path = path + 'benckmark.pickle'
    with open(file_path, 'wb') as file:
        pickle.dump(bm, file)
        
        
    bm_list.append(bm)
    


    

    

Computing neighbors:   0%|          | 0/8 [00:00<?, ?it/s]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12040. The TBB threading layer is disabled.
  warnings.warn(problem)
Metrics:  60%|██████    | 6/10 [01:30<00:45, 11.35s/it, Batch correction: kbet_per_label]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size 

/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a

INFO     gdT CD158b+ consists of a single batch or is too small. Skip.                                             


/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)

Metrics:  70%|███████   | 7/10 [02:09<01:01, 20.46s/it, Batch correction: graph_connectivity]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_graph_connectivity.py:30: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(comps)

Metrics:   0%|          | 0/10 [00:00<?, ?it/s]
                           

/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a

...

INFO     gdT CD158b+ consists of a single batch or is too small. Skip.                                             


/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)

Metrics:  70%|███████   | 7/10 [01:37<00:42, 14.21s/it, Batch correction: graph_connectivity]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_graph_connectivity.py:30: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(comps)

Metrics:   0%|          | 0/10 [00:00<?, ?it/s]
                           

/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a

INFO     gdT CD158b+ consists of a single batch or is too small. Skip.                                             


/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)

Metrics:  70%|███████   | 7/10 [01:32<00:37, 12.48s/it, Batch correction: graph_connectivity]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_graph_connectivity.py:30: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(comps)

Metrics:   0%|          | 0/10 [00:00<?, ?it/s]
                                                                                             
Metrics:  60%|██████    | 6/10 [01:04<00:23,  5.94s/it, Batch correction: kbet_per_label]/projectnb/czproj/ycliu137/.conda/envs/scanpy_env/lib/python3.9/site-packages/scib_metrics/_kbet.py:181: FutureWarning

# Visualization

In [ ]:
for dataset in datasets:
    
    path = OUTPUT_PATH + dataset + '/'
    with open(path + 'benckmark.pickle', 'rb') as file:
        bm = pickle.load(file)
        
    bm.plot_results_table_new = plot_results_table_new
    
    print(dataset+': ')
    bm.plot_results_table_new(self=bm, min_max_scale=False, 
                              save_fig= OUTPUT_PATH + 'bm_plots/bm_plot_' + dataset + '.pdf')

In [ ]:
for dataset in datasets:
    
    path = OUTPUT_PATH + dataset + '/'
    with open(path + 'benckmark.pickle', 'rb') as file:
        bm = pickle.load(file)
    
    print(dataset+': ')
    
    df = bm.get_results(min_max_scale=True, clean_names=True)
    
    path_csv = OUTPUT_PATH + 'bm_csv/bm_csv_' + dataset + '.csv'
    df.to_csv(path_csv, index=True, header=True)
